In [ ]:
import random
import networkx as nx
import numpy as np
import math
import matplotlib.pyplot as plt

N = 100  # performance of algorithm
V = set(range(N + 1))
location = [(np.random.uniform(0, 10), np.random.uniform(0, 10))]
plt.plot(location[-1][0], location[-1][1], 'rs')

d = [0]
for i in V - {0}:
    location.append((np.random.uniform(0, 10), np.random.uniform(0, 10)))
    d.append(random.randint(1, 3))
    plt.scatter(location[-1][0], location[-1][1], d[-1] * 20, 'b')
    plt.text(location[-1][0], location[-1][1], i)

plt.show()


def distance(a, b):
    return math.sqrt((a[0] - b[0]) * (a[0] - b[0]) + (a[0] - b[0]) * (a[0] - b[0]))


c = [[0 if i == j else distance(location[i], location[j]) for j in V] for i in V]

Capacity = 8  # capacity of truck
M = math.ceil(sum(d) / Capacity)

print(M)  # illustration of the problem


def create_individual():
    """
    Create a random individual for the genetic algorithm.
    Each individual represents a possible solution - a permutation of the points to visit.
    """
    individual = list(V - {0})
    random.shuffle(individual)
    return individual


def evaluate_fitness(individual):
    """
    Evaluate the fitness value of an individual.
    The fitness value is the total distance traveled by all trucks to visit all points.
    """
    fitness = 0
    truck_capacity = Capacity
    truck_location = location[0]
    for point in individual:
        point_location = location[point]
        fitness += distance(truck_location, point_location)
        truck_location = point_location
        truck_capacity -= d[point]
        if truck_capacity < 0:
            fitness += distance(truck_location, location[0])
            truck_location = location[0]
            truck_capacity = Capacity
    fitness += distance(truck_location, location[0])  # Return to depot
    return fitness


def crossover(parent1, parent2):
    """
    Perform crossover between two parents to produce offspring.
    Ordered crossover (OX) is used to preserve the relative order of points.
    """
    length = len(parent1)
    start = random.randint(0, length - 1)
    end = random.randint(start + 1, length)
    offspring = [-1] * length

    # Copy the segment between start and end from parent 1 to the offspring
    offspring[start:end] = parent1[start:end]

    # Fill in the remaining positions with the genes from parent 2
    pointer = end
    for gene in parent2[end:] + parent2[:end]:
        if gene not in offspring:
            offspring[pointer % length] = gene
            pointer = (pointer + 1) % length

    return offspring


def mutate(individual, mutation_rate):
    """
    Perform mutation on an individual.
    Swap mutation is used to swap two points in the individual.
    """
    if random.random() < mutation_rate:
        index1, index2 = random.sample(range(len(individual)), 2)
        individual[index1], individual[index2] = individual[index2], individual[index1]
    return individual


def genetic_algorithm(population_size, generations, crossover_rate, mutation_rate):
    """
    Perform the genetic algorithm to find the optimal solution.
    """
    population = [create_individual() for _ in range(population_size)]
    best_fitness = float('inf')
    best_individual = None

    for generation in range(generations):
        offspring = []
        for _ in range(population_size // 2):
            parent1, parent2 = random.choices(population, k=2)
            if random.random() < crossover_rate:
                child1 = crossover(parent1, parent2)
                child2 = crossover(parent2, parent1)
                offspring.extend([child1, child2])
            else:
                offspring.extend([parent1, parent2])

        population = offspring
        for individual in population:
            individual = mutate(individual, mutation_rate)
            fitness = evaluate_fitness(individual)
            if fitness < best_fitness:
                best_fitness = fitness
                best_individual = individual

        if generation % 10 == 0:
            print(f"Generation {generation}: Best Fitness = {best_fitness}")

    return best_individual, best_fitness


# Parameters for the genetic algorithm
population_size = 100
generations = 200
crossover_rate = 0.8
mutation_rate = 0.2

best_individual, best_fitness = genetic_algorithm(population_size, generations, crossover_rate, mutation_rate)
print("Best Individual:", best_individual)
print("Best Fitness:", best_fitness)